In [22]:
from datetime import datetime, timedelta

# News in now-timespan will be kept in database 
now = datetime.now()
timespan = timedelta(days=30)

In [19]:
# newsapi.ai key: 0215dfd8-3e70-4f43-868f-6991882ac84f
# 100 items = 1 token, 2000 total tokens per month
from eventregistry import *

er = EventRegistry(apiKey = '0215dfd8-3e70-4f43-868f-6991882ac84f')
query = {
    "$query": {
        "$and": [
            {
                "categoryUri": "news/Politics"
            },
            {
                "dateStart": "2023-11-19",
                "dateEnd": "2023-11-26",
#                 "dateStart": f"{(now.date()-timedelta(days=1)).strftime("%Y-%m-%d")}",
#                 "dateEnd": f"{now.date().strftime("%Y-%m-%d")}",
                "lang": "eng"
            }
        ]
    },
    "$filter": {
        "startSourceRankPercentile": 0,
        "endSourceRankPercentile": 10
    }
}
q = QueryArticlesIter.initWithComplexQuery(query)
# change maxItems to get the number of results that you want
articles = []
for article in q.execQuery(er, maxItems=10000):
    articles.append(article)

In [23]:
import json

with open('news_database_dict.json', 'r') as dict_json_file:
    news_database_dict = json.load(dict_json_file)
with open('news_database.json', 'r') as data_json_file:
    news_database = json.load(data_json_file)
for a in articles:
    if a['title'] not in news_database_dict.keys():
        news_database_dict[a['title']] = [a['date'], a['title'], a['source'], a['url'], a['body']]
        news_database.append({'date':a['date'], 
                              'title':a['title'], 
                              'source_url':a['source']['uri'], 
                              'source_name':a['source']['title'], 
                              'url':a['url'], 
                              'body':a['body']})
for key in news_database_dict: # Remove old news
    if datetime.strptime(news_database_dict[key][0], '%Y-%m-%d') < now - timespan: 
        news_database[:] = [e for e in news_database if e['title'] != key]
        del news_database_dict[key]
with open('news_database_dict.json', 'w') as out_dict:
    json.dumps(news_database_dict, out_dict, indent=4)
with open('news_database.json', 'w') as outfile:
    json.dumps(news_database, outfile, indent=4)
